# ETL Process to clean data before analysis

## Preparing environment

In [29]:
import pandas as pd

In [30]:
import sys
sys.path.append('../high_performance_employee_resign_prediction')
from utils import paths

## Data Dictionary

* id_employee - Employee ID
* id_last_boss - Boss ID
* seniority - 1: For people who don't have anyone in charge, 2: For those who have people in charge
* work_modality - Work modality specified on employee contract
* office_distance -  Distance in kilometers from employee home to work.
* low_health_days -  Number of days pf justified sick leave since employee started to work for the company.
* gender - Employee gender
* recruitment_channel - Indicates the means by which the application process took place. 
* average_permanence - Mean time in years that the employee has worked in previous laboral experience.
* birth_date - Birth date in format dd/mm/yyyy.
* salary - It is the amount corresponding to the monthly remuneration received.
* psi_score - Is the score obtained in psychometric test during selection process.
* join_date - Date when the employee started to work for the company.
* marital_estatus - Marital Status.
* performance_score - Is the score obtained by the employee it his last quarterly performance test if he had.
* resign - "0" if employee continues working or "1" if employee left the company on his first 6 months.

## Extracting data

In [31]:
train_df = pd.read_csv(paths.data_raw_dir('train_data.csv'), sep=';')

test_df = pd.read_csv(paths.data_raw_dir('test_data.csv'), sep=';')

## Checking data

In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_employee          2152 non-null   int64  
 1   id_last_boss         2061 non-null   float64
 2   seniority            2152 non-null   int64  
 3   work_modality        2152 non-null   object 
 4   office_distance      2152 non-null   float64
 5   low_health_days      2152 non-null   int64  
 6   gender               2152 non-null   object 
 7   recruitment_channel  2152 non-null   object 
 8   average_permanence   2152 non-null   int64  
 9   birth_date           2152 non-null   object 
 10  salary               2152 non-null   int64  
 11  performance_score    2084 non-null   float64
 12  psi_score            2152 non-null   int64  
 13  join_date            2152 non-null   object 
 14  marital_estatus      2152 non-null   object 
 15  resign               2152 non-null   i

In [33]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_employee          2020 non-null   int64  
 1   id_last_boss         1937 non-null   float64
 2   seniority            2020 non-null   int64  
 3   work_modality        2020 non-null   object 
 4   office_distance      2020 non-null   float64
 5   low_health_days      2020 non-null   int64  
 6   gender               2020 non-null   object 
 7   recruitment_channel  2020 non-null   object 
 8   average_permanence   2020 non-null   int64  
 9   birth_date           2020 non-null   object 
 10  salary               2020 non-null   int64  
 11  performance_score    2020 non-null   int64  
 12  psi_score            2020 non-null   int64  
 13  join_date            2020 non-null   object 
 14  marital_estatus      2020 non-null   object 
dtypes: float64(2), int64(7), object(6)
mem

In [34]:
train_df.describe()

,id_employee,id_last_boss,seniority,office_distance,low_health_days,average_permanence,salary,performance_score,psi_score,resign
count,2152.000000,2061.000000,2152.000000,2152.000000,2152.000000,2152.000000,2.152000e+03,2084.000000,2152.000000,2152.000000
mean,102070.160781,102090.326055,1.042286,3.111768,2.709108,6.644981,4.570211e+05,63.472169,75.286245,0.466078
std,1210.999342,43.586010,0.201288,1.789050,2.982975,4.283794,3.029435e+05,23.110594,6.050987,0.498964
min,100001.000000,102000.000000,1.000000,0.120000,0.000000,1.000000,7.652100e+04,5.000000,58.000000,0.000000
25%,101028.750000,102054.000000,1.000000,1.818750,1.000000,3.000000,2.604398e+05,42.000000,71.000000,0.000000
50%,102048.500000,102091.000000,1.000000,2.510000,2.000000,6.000000,3.734225e+05,65.000000,75.000000,0.000000
75%,103135.500000,102127.000000,1.000000,4.151250,3.000000,9.000000,6.741930e+05,90.000000,79.000000,1.000000
max,104171.000000,102172.000000,2.000000,21.050000,35.000000,25.000000,1.900000e+06,99.000000,98.000000,1.000000


In [35]:
test_df.describe()

,id_employee,id_last_boss,seniority,office_distance,low_health_days,average_permanence,salary,performance_score,psi_score
count,2020.000000,1937.000000,2020.000000,2020.000000,2020.000000,2020.000000,2.020000e+03,2020.000000,2020.00000
mean,102103.807921,102089.646877,1.041089,3.059394,2.662376,6.713366,4.638236e+05,89.146040,75.44505
std,1197.751058,43.691244,0.198545,1.637131,2.817351,4.307580,3.063601e+05,4.851837,6.00944
min,100000.000000,102000.000000,1.000000,0.110000,0.000000,1.000000,7.551700e+04,80.000000,58.00000
25%,101071.250000,102052.000000,1.000000,1.805000,1.000000,3.000000,2.663110e+05,85.000000,71.00000
50%,102117.500000,102089.000000,1.000000,2.542500,2.000000,6.000000,3.747200e+05,90.000000,76.00000
75%,103126.250000,102126.000000,1.000000,4.000000,3.000000,9.000000,6.763075e+05,92.000000,79.00000
max,104172.000000,102172.000000,2.000000,14.045000,23.000000,27.000000,1.900000e+06,99.000000,98.00000


Observing the statistics summary, we can see that there are some outliers in `office_distance`, `low_health_days`, `average_permanence` and `salary` for both train and test data. They will be analyzed and actions will be taken during the EDA.

## Looking for typo errors in object data

In [36]:
train_df.describe(include='object')

,work_modality,gender,recruitment_channel,birth_date,join_date,marital_estatus
count,2152,2152,2152,2152,2152,2152
unique,2,2,5,1980,1595,4
top,Presencial,Hombre,Portal Web,24/11/1978,3/10/2013,Soltero
freq,1530,1095,986,3,5,925


In [37]:
test_df.describe(include='object')

,work_modality,gender,recruitment_channel,birth_date,join_date,marital_estatus
count,2020,2020,2020,2020,2020,2020
unique,2,2,5,1883,1527,4
top,Presencial,Mujer,Portal Web,27/04/1972,29/09/2017,Soltero
freq,1408,1037,869,3,5,800


In [38]:
# Checking work_modality categories

print(train_df.work_modality.unique())
print(test_df.work_modality.unique())

['Híbrida' 'Presencial']
['Híbrida' 'Presencial']


In [39]:
# Checking recruitment_channel categories

print(train_df.recruitment_channel.unique())
print(test_df.recruitment_channel.unique())

['Ferias & Networking' 'Referidos' 'Linkedin' 'Portal Web' 'Headhunter']
['Ferias & Networking' 'Portal Web' 'Linkedin' 'Referidos' 'Headhunter']


In [40]:
# Checking marital_estatus categories

print(train_df.marital_estatus.unique())
print(test_df.marital_estatus.unique())

['Soltero' 'Viudo' 'Divorciado' 'Casado']
['Soltero' 'Casado' 'Viudo' 'Divorciado']


From the data dictionary we can see that `seniority` and `id_last_boss` are categories too. However, id_ultimo_jefe has many categories, so it will be analyzed during the hypothesis testing phase.

In [41]:
cat_cols = ['work_modality', 'gender', 'recruitment_channel', 'marital_estatus', 'seniority']

# Defining a function to check categories proportion

def check_categories_proportion(df, col):
    
    """
    Calculate and print the proportion of each category in a specified column of a DataFrame.

    This function takes a pandas DataFrame and a column name, calculates the proportion 
    (as a percentage) of each unique value (category) in the specified column, and prints 
    the resulting proportions.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.
    col (str): The name of the column for which to calculate category proportions.

    Returns:
    None: This function prints the category proportions and does not return any value.

    Example:
    >>> import pandas as pd
    >>> data = {'category': ['A', 'B', 'A', 'C', 'B', 'A']}
    >>> df = pd.DataFrame(data)
    >>> check_categories_proportion(df, 'category')
    A    50.0
    B    33.3
    C    16.7
    Name: category, dtype: float64
    """
    
    proportion = df[col].value_counts(normalize=True) * 100
    print(proportion)

In [42]:
for col in cat_cols:
    check_categories_proportion(train_df, col)
    print('-'*40)

work_modality
Presencial    71.096654
Híbrida       28.903346
Name: proportion, dtype: float64
----------------------------------------
gender
Hombre    50.8829
Mujer     49.1171
Name: proportion, dtype: float64
----------------------------------------
recruitment_channel
Portal Web             45.817844
Linkedin               22.676580
Referidos              15.706320
Ferias & Networking    14.312268
Headhunter              1.486989
Name: proportion, dtype: float64
----------------------------------------
marital_estatus
Soltero       42.983271
Casado        38.708178
Divorciado    10.315985
Viudo          7.992565
Name: proportion, dtype: float64
----------------------------------------
seniority
1    95.771375
2     4.228625
Name: proportion, dtype: float64
----------------------------------------


In [43]:
for col in cat_cols:
    check_categories_proportion(test_df, col)
    print('-'*40)

work_modality
Presencial    69.70297
Híbrida       30.29703
Name: proportion, dtype: float64
----------------------------------------
gender
Mujer     51.336634
Hombre    48.663366
Name: proportion, dtype: float64
----------------------------------------
recruitment_channel
Portal Web             43.019802
Linkedin               25.247525
Referidos              16.336634
Ferias & Networking    14.059406
Headhunter              1.336634
Name: proportion, dtype: float64
----------------------------------------
marital_estatus
Soltero       39.603960
Casado        38.514851
Divorciado    12.425743
Viudo          9.455446
Name: proportion, dtype: float64
----------------------------------------
seniority
1    95.891089
2     4.108911
Name: proportion, dtype: float64
----------------------------------------


Categories proportion in both train and test data are similar.

## Extracting info from date columns

In [44]:
# Converting date columns to dtype datetime

train_df['join_date'] = pd.to_datetime(train_df['join_date'], dayfirst=True)
train_df['birth_date'] = pd.to_datetime(train_df['birth_date'], dayfirst=True)

test_df['join_date'] = pd.to_datetime(test_df['join_date'], dayfirst=True)
test_df['birth_date'] = pd.to_datetime(test_df['birth_date'], dayfirst=True)

In [45]:
# Calculating age of join

train_df['join_age'] = (train_df['join_date'] - train_df['birth_date']).dt.days // 365

test_df['join_age'] = (test_df['join_date'] - test_df['birth_date']).dt.days // 365

In [46]:
# Obtaining season info

train_df['join_year'] = train_df['join_date'].dt.year
train_df['join_month'] = train_df['join_date'].dt.month

test_df['join_year'] = test_df['join_date'].dt.year
test_df['join_month'] = test_df['join_date'].dt.month

In [47]:
# Discarding original dates

train_df.drop(['birth_date', 'join_date'], axis=1, inplace=True)
test_df.drop(['birth_date', 'join_date'], axis=1, inplace=True)

In [48]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_employee          2152 non-null   int64  
 1   id_last_boss         2061 non-null   float64
 2   seniority            2152 non-null   int64  
 3   work_modality        2152 non-null   object 
 4   office_distance      2152 non-null   float64
 5   low_health_days      2152 non-null   int64  
 6   gender               2152 non-null   object 
 7   recruitment_channel  2152 non-null   object 
 8   average_permanence   2152 non-null   int64  
 9   salary               2152 non-null   int64  
 10  performance_score    2084 non-null   float64
 11  psi_score            2152 non-null   int64  
 12  marital_estatus      2152 non-null   object 
 13  resign               2152 non-null   int64  
 14  join_age             2152 non-null   int64  
 15  join_year            2152 non-null   i

In [49]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_employee          2020 non-null   int64  
 1   id_last_boss         1937 non-null   float64
 2   seniority            2020 non-null   int64  
 3   work_modality        2020 non-null   object 
 4   office_distance      2020 non-null   float64
 5   low_health_days      2020 non-null   int64  
 6   gender               2020 non-null   object 
 7   recruitment_channel  2020 non-null   object 
 8   average_permanence   2020 non-null   int64  
 9   salary               2020 non-null   int64  
 10  performance_score    2020 non-null   int64  
 11  psi_score            2020 non-null   int64  
 12  marital_estatus      2020 non-null   object 
 13  join_age             2020 non-null   int64  
 14  join_year            2020 non-null   int32  
 15  join_month           2020 non-null   i

## Missing values treatment

From context of the business, is well known that people with performance_score greater than or equal to 80 is considered people with high perfomance, and everyone else is considered low performance. There are some missing values in this column on train data, and observing the statistics summary, most of the employees from the train data are low performance, while all the employees in the test data are high performance; this lead us to impute those missing values with 80, in order to help the people to generalize better the high performance employees. Additionally, performance categorical column will be added with 'high' for employees with performance score greater or equal to 80 and 'low' for everyone else.

In [50]:
train_df['performance_score'] = train_df['performance_score'].fillna(80)

Since id_ultimo_jefe is a categorical column, we can't impute it. So we gonna fill missing values with bfill method.

In [51]:
train_df['id_last_boss'] = train_df['id_last_boss'].fillna(method='bfill')

test_df['id_last_boss'] = test_df['id_last_boss'].fillna(method='bfill')

C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_19320\2512515177.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_df['id_last_boss'] = train_df['id_last_boss'].fillna(method='bfill')
C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_19320\2512515177.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_df['id_last_boss'] = test_df['id_last_boss'].fillna(method='bfill')


## Changing id_last_boss dtype

In [52]:
train_df['id_last_boss'] = train_df['id_last_boss'].astype(int)

test_df['id_last_boss'] = test_df['id_last_boss'].astype(int)

In [53]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_employee          2152 non-null   int64  
 1   id_last_boss         2152 non-null   int32  
 2   seniority            2152 non-null   int64  
 3   work_modality        2152 non-null   object 
 4   office_distance      2152 non-null   float64
 5   low_health_days      2152 non-null   int64  
 6   gender               2152 non-null   object 
 7   recruitment_channel  2152 non-null   object 
 8   average_permanence   2152 non-null   int64  
 9   salary               2152 non-null   int64  
 10  performance_score    2152 non-null   float64
 11  psi_score            2152 non-null   int64  
 12  marital_estatus      2152 non-null   object 
 13  resign               2152 non-null   int64  
 14  join_age             2152 non-null   int64  
 15  join_year            2152 non-null   i

In [54]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_employee          2020 non-null   int64  
 1   id_last_boss         2020 non-null   int32  
 2   seniority            2020 non-null   int64  
 3   work_modality        2020 non-null   object 
 4   office_distance      2020 non-null   float64
 5   low_health_days      2020 non-null   int64  
 6   gender               2020 non-null   object 
 7   recruitment_channel  2020 non-null   object 
 8   average_permanence   2020 non-null   int64  
 9   salary               2020 non-null   int64  
 10  performance_score    2020 non-null   int64  
 11  psi_score            2020 non-null   int64  
 12  marital_estatus      2020 non-null   object 
 13  join_age             2020 non-null   int64  
 14  join_year            2020 non-null   int32  
 15  join_month           2020 non-null   i

## Adding performance column

In [55]:
train_df['performance'] = ['high' if score >= 80 else 'low' for score in train_df['performance_score']]
test_df['performance'] = ['high' if score >= 80 else 'low' for score in test_df['performance_score']]

## Adding new features

Boss id are also present in employee id, so we can use this information to create new features that can help to improve the results

In [66]:
expanded_train_df = pd.merge(train_df, train_df.drop(columns=['resign']), how='left',
                             left_on='id_last_boss', right_on='id_employee',
                             suffixes=('_employee', '_boss'))

In [67]:
expanded_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2152 non-null   int64  
 1   id_last_boss_employee         2152 non-null   int32  
 2   seniority_employee            2152 non-null   int64  
 3   work_modality_employee        2152 non-null   object 
 4   office_distance_employee      2152 non-null   float64
 5   low_health_days_employee      2152 non-null   int64  
 6   gender_employee               2152 non-null   object 
 7   recruitment_channel_employee  2152 non-null   object 
 8   average_permanence_employee   2152 non-null   int64  
 9   salary_employee               2152 non-null   int64  
 10  performance_score_employee    2152 non-null   float64
 11  psi_score_employee            2152 non-null   int64  
 12  marital_estatus_employee      2152 non-null   object 
 13  res

In [70]:
expanded_test_df = pd.merge(test_df, test_df, how='left',
                             left_on='id_last_boss', right_on='id_employee',
                             suffixes=('_employee', '_boss'))

In [84]:
expanded_test_df[['id_employee_employee', 'id_last_boss_employee', 'id_employee_boss', 'id_last_boss_boss']]

,id_employee_employee,id_last_boss_employee,id_employee_boss,id_last_boss_boss
0,100486,102115,102115.0,102116.0
1,103752,102074,102074.0,102028.0
2,103937,102150,102150.0,102085.0
3,101744,102172,NaN,NaN
4,101037,102060,102060.0,102041.0
...,...,...,...,...
2015,103755,102161,102161.0,102039.0
2016,103976,102171,NaN,NaN
2017,104115,102144,NaN,NaN
2018,103920,102152,NaN,NaN


In [81]:
expanded_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2020 non-null   int64  
 1   id_last_boss_employee         2020 non-null   int32  
 2   seniority_employee            2020 non-null   int64  
 3   work_modality_employee        2020 non-null   object 
 4   office_distance_employee      2020 non-null   float64
 5   low_health_days_employee      2020 non-null   int64  
 6   gender_employee               2020 non-null   object 
 7   recruitment_channel_employee  2020 non-null   object 
 8   average_permanence_employee   2020 non-null   int64  
 9   salary_employee               2020 non-null   int64  
 10  performance_score_employee    2020 non-null   int64  
 11  psi_score_employee            2020 non-null   int64  
 12  marital_estatus_employee      2020 non-null   object 
 13  joi

## Missing Values treatment

In order to impute missing values from new features from boss, let's make some notations and assumptions:

- id_employee_boss, id_last_boss_boss, marital_estatus_boss, join_month_boss are not relevant
- seniority_boss is not relevant since all boss have seniority 2
- work_modality_boss, recruitment_channel_boss will be assumed to be the same from employee
- gender_boss will be assumed as no relevant
- Numerical features will be imputed using KNN Imputer
- performance_boss will be determined after imputing numerical values

In [89]:
y = expanded_train_df['resign']

In [91]:
# Dropping columns

expanded_train_df.drop(columns=['id_employee_boss', 'id_last_boss_boss',
                                'marital_estatus_boss', 'join_month_boss',
                                'seniority_boss', 'gender_boss', 'performance_boss'], inplace=True)

expanded_test_df.drop(columns=['id_employee_boss', 'id_last_boss_boss',
                                'marital_estatus_boss', 'join_month_boss',
                                'seniority_boss', 'gender_boss', 'performance_boss'], inplace=True)

In [93]:
# Dropping resign

expanded_train_df.drop(columns=['resign'], inplace=True)

In [99]:
# Imputing missing values from work_modality_boss and recruitment_channel_boss

no_miss_train = expanded_train_df.dropna()
miss_train = expanded_train_df[expanded_train_df.isna().any(axis=1)]

miss_train['work_modality_boss'] = miss_train['work_modality_employee']
miss_train['recruitment_channel_boss'] = miss_train['recruitment_channel_employee']

expanded_train_df_clean = pd.concat([no_miss_train, miss_train], axis=0).sort_index()

C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_19320\338459526.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miss_train['work_modality_boss'] = miss_train['work_modality_employee']
C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_19320\338459526.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miss_train['recruitment_channel_boss'] = miss_train['recruitment_channel_employee']


In [102]:
# Imputing missing values from work_modality_boss and recruitment_channel_boss

no_miss_test = expanded_test_df.dropna()
miss_test = expanded_test_df[expanded_test_df.isna().any(axis=1)]

miss_test['work_modality_boss'] = miss_test['work_modality_employee']
miss_test['recruitment_channel_boss'] = miss_test['recruitment_channel_employee']

expanded_test_df_clean = pd.concat([no_miss_test, miss_test], axis=0).sort_index()

C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_19320\1881566490.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miss_test['work_modality_boss'] = miss_test['work_modality_employee']
C:\Users\WILLIAM\AppData\Local\Temp\ipykernel_19320\1881566490.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miss_test['recruitment_channel_boss'] = miss_test['recruitment_channel_employee']


In [106]:
# Preparing missing values imputation

from sklearn.preprocessing import OneHotEncoder, PowerTransformer
from sklearn.impute import KNNImputer

In [108]:
# Saving ids

train_ids = expanded_train_df_clean[['id_employee_employee', 'id_last_boss_employee']]
test_ids = expanded_test_df_clean[['id_employee_employee', 'id_last_boss_employee']]

In [109]:
# Dropping columns

expanded_train_df_clean.drop(columns=['id_employee_employee', 'id_last_boss_employee'], inplace=True)
expanded_test_df_clean.drop(columns=['id_employee_employee', 'id_last_boss_employee'], inplace=True)

In [111]:
expanded_train_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2152 entries, 0 to 2151
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   seniority_employee            2152 non-null   int64  
 1   work_modality_employee        2152 non-null   object 
 2   office_distance_employee      2152 non-null   float64
 3   low_health_days_employee      2152 non-null   int64  
 4   gender_employee               2152 non-null   object 
 5   recruitment_channel_employee  2152 non-null   object 
 6   average_permanence_employee   2152 non-null   int64  
 7   salary_employee               2152 non-null   int64  
 8   performance_score_employee    2152 non-null   float64
 9   psi_score_employee            2152 non-null   int64  
 10  marital_estatus_employee      2152 non-null   object 
 11  join_age_employee             2152 non-null   int64  
 12  join_year_employee            2152 non-null   int32  
 13  join_mon

In [115]:
# Defining categorical and numerical columns

cat_cols = ['seniority_employee', 'work_modality_employee', 'gender_employee', 'recruitment_channel_employee',
            'marital_estatus_employee', 'join_year_employee', 'join_month_employee', 'performance_employee', 
            'work_modality_boss', 'recruitment_channel_boss']

num_cols = ['office_distance_employee', 'low_health_days_employee', 'average_permanence_employee', 'salary_employee',
            'performance_score_employee', 'psi_score_employee', 'join_age_employee', 'office_distance_boss', 'low_health_days_boss',
            'average_permanence_boss', 'salary_boss', 'performance_score_boss', 'psi_score_boss', 'join_age_boss', 'join_year_boss']

In [118]:
# Encoding categorical columns

oh_encoder = OneHotEncoder(sparse_output=False, drop='first')

train_clean_encoded = pd.DataFrame(oh_encoder.fit_transform(expanded_train_df_clean[cat_cols]))
train_clean_encoded.columns = oh_encoder.get_feature_names_out(cat_cols)

test_clean_encoded = pd.DataFrame(oh_encoder.transform(expanded_test_df_clean[cat_cols]))
test_clean_encoded.columns = oh_encoder.get_feature_names_out(cat_cols)

In [124]:
# Transforming numerical columns

pt = PowerTransformer(method='yeo-johnson')

train_clean_scaled = pd.DataFrame(pt.fit_transform(expanded_train_df_clean[num_cols]), columns=num_cols)

test_clean_scaled = pd.DataFrame(pt.transform(expanded_test_df_clean[num_cols]), columns=num_cols)

In [125]:
# Preparing df to make numerical imputation

train_clean_final = pd.concat([train_clean_encoded, train_clean_scaled], axis=1)
test_clean_final = pd.concat([test_clean_encoded, test_clean_scaled], axis=1)

In [127]:
# Imputing missing values

knn_imp = KNNImputer()

train_clean_final = pd.DataFrame(knn_imp.fit_transform(train_clean_final), columns=train_clean_final.columns)
test_clean_final = pd.DataFrame(knn_imp.transform(test_clean_final), columns=train_clean_final.columns)

In [128]:
train_clean_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 53 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   seniority_employee_2                     2152 non-null   float64
 1   work_modality_employee_Presencial        2152 non-null   float64
 2   gender_employee_Mujer                    2152 non-null   float64
 3   recruitment_channel_employee_Headhunter  2152 non-null   float64
 4   recruitment_channel_employee_Linkedin    2152 non-null   float64
 5   recruitment_channel_employee_Portal Web  2152 non-null   float64
 6   recruitment_channel_employee_Referidos   2152 non-null   float64
 7   marital_estatus_employee_Divorciado      2152 non-null   float64
 8   marital_estatus_employee_Soltero         2152 non-null   float64
 9   marital_estatus_employee_Viudo           2152 non-null   float64
 10  join_year_employee_2013                  2152 no

In [129]:
test_clean_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 53 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   seniority_employee_2                     2020 non-null   float64
 1   work_modality_employee_Presencial        2020 non-null   float64
 2   gender_employee_Mujer                    2020 non-null   float64
 3   recruitment_channel_employee_Headhunter  2020 non-null   float64
 4   recruitment_channel_employee_Linkedin    2020 non-null   float64
 5   recruitment_channel_employee_Portal Web  2020 non-null   float64
 6   recruitment_channel_employee_Referidos   2020 non-null   float64
 7   marital_estatus_employee_Divorciado      2020 non-null   float64
 8   marital_estatus_employee_Soltero         2020 non-null   float64
 9   marital_estatus_employee_Viudo           2020 non-null   float64
 10  join_year_employee_2013                  2020 no

In [146]:
# Making inverse transformations

train_cat_cols = pd.DataFrame(oh_encoder.inverse_transform(train_clean_final[train_clean_encoded.columns]), columns=cat_cols)
test_cat_cols = pd.DataFrame(oh_encoder.inverse_transform(test_clean_final[train_clean_encoded.columns]), columns=cat_cols)

train_num_cols = pd.DataFrame(pt.inverse_transform(train_clean_final[num_cols]), columns=num_cols)
test_num_cols = pd.DataFrame(pt.inverse_transform(test_clean_final[num_cols]), columns=num_cols)

From original data it's known that all numerical columns but office_distance are integers.

In [148]:
# Changing dtypes

float_cols = ['office_distance_employee', 'office_distance_boss']
int_cols = [col for col in num_cols if col not in float_cols]

train_num_cols[int_cols] = train_num_cols[int_cols].astype(int)
test_num_cols[int_cols] = test_num_cols[int_cols].astype(int)

In [150]:
# Concatenating all features

train_final = pd.concat([train_ids, train_cat_cols, train_num_cols, y], axis=1)
test_final = pd.concat([test_ids, test_cat_cols, test_num_cols], axis=1)

In [151]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2152 entries, 0 to 2151
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2152 non-null   int64  
 1   id_last_boss_employee         2152 non-null   int32  
 2   seniority_employee            2152 non-null   object 
 3   work_modality_employee        2152 non-null   object 
 4   gender_employee               2152 non-null   object 
 5   recruitment_channel_employee  2152 non-null   object 
 6   marital_estatus_employee      2152 non-null   object 
 7   join_year_employee            2152 non-null   object 
 8   join_month_employee           2152 non-null   object 
 9   performance_employee          2152 non-null   object 
 10  work_modality_boss            2152 non-null   object 
 11  recruitment_channel_boss      2152 non-null   object 
 12  office_distance_employee      2152 non-null   float64
 13  low_heal

In [152]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2020 entries, 0 to 2019
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2020 non-null   int64  
 1   id_last_boss_employee         2020 non-null   int32  
 2   seniority_employee            2020 non-null   object 
 3   work_modality_employee        2020 non-null   object 
 4   gender_employee               2020 non-null   object 
 5   recruitment_channel_employee  2020 non-null   object 
 6   marital_estatus_employee      2020 non-null   object 
 7   join_year_employee            2020 non-null   object 
 8   join_month_employee           2020 non-null   object 
 9   performance_employee          2020 non-null   object 
 10  work_modality_boss            2020 non-null   object 
 11  recruitment_channel_boss      2020 non-null   object 
 12  office_distance_employee      2020 non-null   float64
 13  low_heal

## Creating new features

These new features allow us to make new features such as salary difference, age difference and boss_performance

In [167]:
# Creating new features

train_final['salary_diff'] = train_final['salary_boss'] - train_final['salary_employee']
train_final['age_diff'] = train_final['join_age_boss'] - train_final['join_age_employee']
train_final['performance_boss'] = ['high' if value > 80 else 'low' for value in train_final['performance_score_boss']]

test_final['salary_diff'] = train_final['salary_boss'] - train_final['salary_employee']
test_final['age_diff'] = train_final['join_age_boss'] - train_final['join_age_employee']
test_final['performance_boss'] = ['high' if value > 80 else 'low' for value in test_final['performance_score_boss']]

In [170]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2152 entries, 0 to 2151
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2152 non-null   int64  
 1   id_last_boss_employee         2152 non-null   int32  
 2   seniority_employee            2152 non-null   object 
 3   work_modality_employee        2152 non-null   object 
 4   gender_employee               2152 non-null   object 
 5   recruitment_channel_employee  2152 non-null   object 
 6   marital_estatus_employee      2152 non-null   object 
 7   join_year_employee            2152 non-null   object 
 8   join_month_employee           2152 non-null   object 
 9   performance_employee          2152 non-null   object 
 10  work_modality_boss            2152 non-null   object 
 11  recruitment_channel_boss      2152 non-null   object 
 12  office_distance_employee      2152 non-null   float64
 13  low_heal

In [171]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2020 entries, 0 to 2019
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_employee_employee          2020 non-null   int64  
 1   id_last_boss_employee         2020 non-null   int32  
 2   seniority_employee            2020 non-null   object 
 3   work_modality_employee        2020 non-null   object 
 4   gender_employee               2020 non-null   object 
 5   recruitment_channel_employee  2020 non-null   object 
 6   marital_estatus_employee      2020 non-null   object 
 7   join_year_employee            2020 non-null   object 
 8   join_month_employee           2020 non-null   object 
 9   performance_employee          2020 non-null   object 
 10  work_modality_boss            2020 non-null   object 
 11  recruitment_channel_boss      2020 non-null   object 
 12  office_distance_employee      2020 non-null   float64
 13  low_heal

## Saving clean data

In [169]:
train_final.to_csv(paths.data_interim_dir('train_clean.csv'), index=False, sep=',')

test_final.to_csv(paths.data_interim_dir('test_clean.csv'), index=False, sep=',')